In [1]:
# Install uncommon packages (will supply requements.txt later)
#!pip install git+https://github.com/boudinfl/pke.git
#!pip install wordninja
#!pip install pyenchant
#!pip install --no-cache-dir pycorpora


import pandas as pd
import numpy as np
import os
import json
import wordninja
from os import listdir
import re
import sys
import ast
import pke
import enchant
import nltk
import string
#import pycorpora
nltk.download('averaged_perceptron_tagger')
d = enchant.Dict("en_US") # ENGLISH Dictionary

# CHANGE THE PATH TO A DATASET HERE
basedir = os.path.join(os.sep, "Users", "Artemiy", "Disk-O", "tyomkolton@mail.ru-mailru", "ML")


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Artemiy/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
data = pd.read_csv(os.path.join(basedir, "CB_export", "cb_organization_descriptions.csv"), index_col="uuid")
#data.iloc[:,1].to_csv('CB_Export_17_08_07/descr.txt')
print("loaded descriptions")
data1 = pd.read_csv(os.path.join(basedir, "CB_export", "cb_organizations.csv"), low_memory=False, index_col="uuid")
print("loaded org infos")

loaded descriptions
loaded org infos


In [3]:
# Merge 2 csv files
# we can time to see which is more effective
#result = pd.concat([data, data1], axis=1, sort=False)
result1 = pd.merge(data, data1, how='inner', left_index=True, right_index=True)
print("merged")
# Delete all companies that do not have any descriptions
result1 = result1.dropna(subset=["description"])

merged


In [4]:
#print(((result.shape[0] - result1.shape[0])/result.shape[0]*100), "% of companies do not have any description")

In [5]:
result1.head()

,description,company_name,primary_role,permalink,domain,homepage_url,country_code,state_code,region,city,...,phone,facebook_url,linkedin_url,cb_url,logo_url,profile_image_url,twitter_url,alias,created_at,updated_at
uuid,,,,,,,,,,,,,,,,,,,,,
f5bb580f-d655-cf3f-9ade-eca5b3f2719f,"Android, Apple - iOS, Blackberry, Windows Phon...",VilarikA,company,/organization/vilarika,vilarika.com.br,http://vilarika.com.br/,BRA,NaN,Rio de Janeiro,Belo Horizonte,...,NaN,NaN,NaN,https://www.crunchbase.com/organization/vilarika,https://www.crunchbase.com/organization/vilari...,http://public.crunchbase.com/t_api_images/v141...,NaN,NaN,2014-12-11 06:46:05,2016-09-07 00:03:51.67913
00000aa4-ba42-9b68-a9c3-040c9f3bf9b9,Formel D GmbH is a automotive manufacturer and...,Formel D GmbH,company,/organization/formel-d-gmbh,formeld.com,http://www.formeld.com,DEU,NaN,DEU - Other,Troisdorf,...,+49 2241 9960,https://www.facebook.com/formeld,https://www.linkedin.com/company/formel-d-group,https://www.crunchbase.com/organization/formel...,https://www.crunchbase.com/organization/formel...,http://public.crunchbase.com/t_api_images/v148...,https://www.twitter.com/formeld_es,NaN,2016-06-01 06:58:37.692725,2017-07-18 07:22:19.15864
f5bd38d1-7719-2935-fb6c-defae39f5b93,[iNFoGooL](http://infogool.com) - The Informat...,infogool,company,/organization/infogool,infogool.com,http://infogool.com,NaN,NaN,NaN,NaN,...,8802393009,http://www.facebook.com/infogool,NaN,https://www.crunchbase.com/organization/infogool,https://www.crunchbase.com/organization/infogo...,http://public.crunchbase.com/t_api_images/v139...,https://www.twitter.com/infogool,NaN,2014-04-12 00:10:12,2016-09-08 22:02:05.585875
773adc18-132a-937d-8841-4f833e64dd56,The Jinfeng Gold Mine is an combined open pit ...,Jinfeng Mine,company,/organization/jinfeng-mine,NaN,NaN,CHN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://www.crunchbase.com/organization/jinfen...,NaN,NaN,NaN,NaN,2016-12-07 06:43:06.955155,2016-12-07 06:45:09.14727
f5bdd48a-a09f-2f4c-bd71-4c4207c7731b,Peardoc offers online tools to convert HTML to...,Peardoc Solutions,company,/organization/peardoc-solutions,peardoc.com,http://www.peardoc.com,IND,NaN,Chennai,Chennai,...,NaN,NaN,NaN,https://www.crunchbase.com/organization/peardo...,https://www.crunchbase.com/organization/peardo...,http://public.crunchbase.com/t_api_images/v140...,https://www.twitter.com/pear_doc,NaN,2014-07-17 10:06:03,2016-03-08 02:56:01.230586


In [6]:
len(result1)

353146

In [7]:
# Constuct a list of companies that have descriptions
d_companies_list = list(result1.loc[:, "company_name"])
d_companies_list = [company.lower() for company in d_companies_list]
#d_companies_list = [company for company in d_companies_list if all(letter.isalpha() for letter in company)]
#d_companies_list = [company for company in d_companies_list if not d.check(company)]
len(d_companies_list)

353146

In [8]:
# Construct a list of companies from dataset
companies_list = os.listdir(os.path.join(basedir, "News_1"))

len(companies_list)

19791

In [9]:
# Get rid of ".json" extension and apply uppercase
companies_list = [re.sub(".json", "", company) for company in companies_list]
print(len(companies_list))
# Get rid of companies with names containing anything else than latin letters (as well as containing "_" separating multiple words in names)
companies_list = [company.lower() for company in companies_list if all(letter in string.ascii_letters for letter in company)]
print(len(companies_list))
# Get rid of companies whose full names consist of one single word from english dictionary
companies_list = [company for company in companies_list if not d.check(company.lower())]
print(len(companies_list))
# Tries to separate a name string into multiple words and if succeeds gets rid of the company (a lot of false positives)
# companies_list = [company for company in companies_list if len(wordninja.split(company.lower())) < 2]
# print(len(companies_list))

19791
12381
11175


In [10]:
companies_list = [company for company in companies_list if company.lower() in d_companies_list]
len(companies_list)

9986

In [11]:
# Prepare the data frame of companies descriptions to be used in final data frame with articles
d_company_df = result1
d_company_df.loc[:,'company_name'] = d_company_df.loc[:,'company_name'].str.lower()
#d_company_df = d_company_df[d_company_df['company_name'].str.lower().isin(d_companies_list)]
#d_company_df = d_company_df[d_company_df['company_name'].isin(d_companies_list)]
d_company_df = d_company_df.reset_index().set_index(keys='company_name')#, verify_integrity=True)

# Delete companies outside or dataset of articles
d_company_df = d_company_df[d_company_df.index.isin(companies_list)]

# Delete the companies which have the same name
d_company_df = d_company_df[~d_company_df.index.duplicated(keep=False)]


d_company_df = d_company_df[['uuid', 'description']]
len(d_company_df)

9748

In [12]:
# Save for later use with keywords extraction
if not os.path.exists(os.path.join(basedir, 'temp_data')):
    os.makedirs(os.path.join(basedir, 'temp_data'))
d_company_df.to_csv(os.path.join(basedir, 'temp_data', 'd_company_df.csv'), sep=";")
d_company_df.head()

,uuid,description
company_name,,
nerites,f5c05e57-d563-db35-8210-6f26d6a250d2,"At Nerites, they combine world class technolog..."
biovascular,7539d769-643c-6c26-5339-ec52bd5b81c2,BioVascular is a clinical stage privately held...
transbiotec,f5e29c35-a53e-635d-ce74-4013c11efec9,"TransBiotec, a development stage company, focu..."
fortressware,f1a53b32-a132-1764-9e2d-54e5324e94fc,"Fortressware, Inc. provides solutions to secur..."
songfor,7789da2f-f1dd-a1ac-434b-355c320b4d71,Auf SongFor.com hast du die Möglichkeit Musik ...


In [13]:
### ONE WAY TO EXTRACT KEYWORDS ###

# With PKE Package
tmp = d_company_df

# initialize keyphrase extraction model, here TopicRank
import pke



import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Artemiy/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [14]:
counter = 0
for index, row in d_company_df.iterrows():
    
    
    counter +=1
    #if counter >3:
    #    break
    
    extractor = pke.unsupervised.TopicRank()
    # load the content of the document, here document is expected to be in raw
    # format (i.e. a simple string of text) and preprocessing is carried out using nltk
    s = row["description"]
    #s = ' '.join(s.split())
    #s = "fwfewk w31231dmm! test test hello you piece of mind balls"
    extractor.read_text(s)

    # keyphrase candidate selection, in the case of TopicRank: sequences of nouns
    # and adjectives
    extractor.candidate_selection()

    # candidate weighting, in the case of TopicRank: using a random walk algorithm
    try:
        extractor.candidate_weighting()
        # N-best selection, keyphrases contains the 10 highest scored candidates as
        # (keyphrase, score) tuples
        keyphrases = extractor.get_n_best(n=10, stemming=False)
        keyphrases = sorted(keyphrases, key=lambda x: x[1], reverse=True)

        tmp.loc[index, 'kp_topicrank'] = str(keyphrases)
    
    except ValueError:
        print("Description for company " + index + 
              " (" + row['uuid'] + ")" +
              " is too short. Description: " + index)
        continue

        
        #if None in keyphrases:
        
    
tmp.tail()

Description for company ineomarketing (954b5316-3bcc-6081-d018-fec2c374d395) is too short. Description: ineomarketing
Description for company arrowsight (23bea55a-2e83-2b39-b53a-d8bd28d218e8) is too short. Description: arrowsight
Description for company looklive (6d3b1bed-302a-3331-6f43-87f91ca6a54b) is too short. Description: looklive
Description for company cityspark (f25f78b7-1a86-bf9d-e5da-24a538b81c61) is too short. Description: cityspark
Description for company rentabilities (aa8e1403-1db8-e5bf-6b5c-423bb826ccfd) is too short. Description: rentabilities
Description for company redkix (0510d73a-8709-bcae-5930-f047ee8b9db7) is too short. Description: redkix
Description for company shaser (538439c3-693e-0c54-76e3-f6c4c237370c) is too short. Description: shaser
Description for company bamboostr (83ccded4-e8ca-782f-2ef6-2c5b020fe48a) is too short. Description: bamboostr
Description for company metacert (83d3852a-f60d-2bac-24fe-afdbfb316fe9) is too short. Description: metacert
Descript

,uuid,description,kp_topicrank
company_name,,,
hooja,ff71401a-f5b0-1dd8-2b72-4a9e5d6ecba0,Hooja is a company focused on the mobile realm...,"[('service', 0.0785488995371759), ('hooja', 0...."
baseclick,d140620f-09bb-dd33-baaa-ecffff1d8bc6,baseclick GmbH is a start-up company funded an...,"[('technologies', 0.10249367027116733), ('star..."
biodetego,fef487a3-2505-e176-7bdc-d82db18525dc,"BioDetego is developing, VASPfore, a new bioma...","[('cancer treatment protocols', 0.105445975900..."
socialthreader,ffe12892-05c4-e8e4-d728-d9b54e7b3aeb,SocialThreader increases effectiveness of digi...,"[('campaign performance', 0.1546622347489285),..."
findally,cc693948-abf3-e3dc-f845-3e38ac718aab,Findally is an image-based product engine that...,"[('image-based product engine', 0.168320437934..."


In [15]:
k_company_df = tmp
len(k_company_df)

9748

In [16]:
k_company_df = k_company_df.dropna()
companies_list = k_company_df.index.get_values()
len(k_company_df)

9726

In [17]:
# Save for later use with keywords extraction
if not os.path.exists(os.path.join(basedir, 'temp_data')):
    os.makedirs(os.path.join(basedir, 'temp_data'))
k_company_df.to_csv(os.path.join(basedir, 'temp_data', 'k_company_df.csv'), sep=";")
k_company_df.head()

,uuid,description,kp_topicrank
company_name,,,
nerites,f5c05e57-d563-db35-8210-6f26d6a250d2,"At Nerites, they combine world class technolog...","[('market', 0.09826419107155268), ('technical ..."
biovascular,7539d769-643c-6c26-5339-ec52bd5b81c2,BioVascular is a clinical stage privately held...,"[('classâ€\x9d products', 0.09608849550420087)..."
transbiotec,f5e29c35-a53e-635d-ce74-4013c11efec9,"TransBiotec, a development stage company, focu...","[('systems', 0.09053148884251386), ('developme..."
fortressware,f1a53b32-a132-1764-9e2d-54e5324e94fc,"Fortressware, Inc. provides solutions to secur...","[('access policies', 0.08053299604810367), ('f..."
songfor,7789da2f-f1dd-a1ac-434b-355c320b4d71,Auf SongFor.com hast du die Möglichkeit Musik ...,"[('zu erleben', 0.14652950099242656), ('das ei..."


In [18]:
# Get the list of paths to articles files and their sizes
paths = [(company, os.path.join(basedir, "News_1", "".join([company.upper(), ".json"]))) for company in companies_list]
paths = [path + (os.stat(path[1]).st_size,) for path in paths]
# Sort by sizes
paths = sorted(paths, key=lambda x: x[2], reverse=True)

print(paths[0:10])

[('brightline', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/BRIGHTLINE.json', 195149313), ('wherefor', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/WHEREFOR.json', 176922132), ('musx', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/MUSX.json', 164413600), ('understory', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/UNDERSTORY.json', 158724779), ('crono', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/CRONO.json', 110729460), ('gridco', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/GRIDCO.json', 107924262), ('lookback', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/LOOKBACK.json', 107008921), ('enervault', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/ENERVAULT.json', 100682369), ('energysavvy', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/ENERGYSAVVY.json', 99732850), ('humon', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/HUMON.json', 95443584)]


In [19]:
all_articles = pd.DataFrame()
num_articles = 0

In [ ]:
for index_f, file in enumerate(paths):
    #if index_f > 2:
    #    break
    
    print(file)
    print(index_f)

    try:
        articles = pd.read_json(file[1])
    except ValueError:
        print("empty/faulty file")
        continue

    if "content" not in articles.columns:
        print("no content")
        continue

    articles.loc[:, "company"] = file[0]
    articles.loc[:, "uuid"] = k_company_df.loc[file[0].lower(), "uuid"]
    articles.loc[:, "description"] = k_company_df.loc[file[0].lower(), "description"]

    articles.dropna(subset=["content"], inplace=True)

    texts = articles.loc[:, "content"]
    keyphrases = ast.literal_eval(k_company_df.loc[file[0].lower(), "kp_topicrank"])
    num_articles = num_articles + len(texts)

    # Check whether the any of the keywords exist in the text
    texts = [text 
             if any([keyphrase in text.lower() 
                     for (keyphrase, prob) in (keyphrases)]) and len(list(text)) > 500 
             else np.NaN
            for text in texts]

    articles.loc[:, "content"] = texts
    articles.dropna(subset=["content"], inplace=True)

    all_articles = all_articles.append(articles, ignore_index=True, sort=False)
    
    

('brightline', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/BRIGHTLINE.json', 195149313)
0
('wherefor', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/WHEREFOR.json', 176922132)
1
('musx', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/MUSX.json', 164413600)
2
('understory', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/UNDERSTORY.json', 158724779)
3
('crono', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/CRONO.json', 110729460)
4
('gridco', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/GRIDCO.json', 107924262)
5
('lookback', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/LOOKBACK.json', 107008921)
6
('enervault', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/ENERVAULT.json', 100682369)
7
('energysavvy', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/ENERGYSAVVY.json', 99732850)
8
('humon', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/HUMON.json', 95443584)
9


('cempra', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/CEMPRA.json', 36856905)
83
('anki', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/ANKI.json', 36708221)
84
('gigamon', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/GIGAMON.json', 36112465)
85
('nextbio', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/NEXTBIO.json', 35627205)
86
('hedgeable', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/HEDGEABLE.json', 35457580)
87
('makerbot', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/MAKERBOT.json', 35324158)
88
('redbubble', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/REDBUBBLE.json', 35252044)
89
('affymax', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/AFFYMAX.json', 35241180)
90
('sourcefire', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/SOURCEFIRE.json', 35070584)
91
('twilio', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/TWILIO.json', 35042677)
92
('in

('wildfang', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/WILDFANG.json', 26946155)
165
('reflektion', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/REFLEKTION.json', 26913159)
166
('procom', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/PROCOM.json', 26900683)
167
('moli', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/MOLI.json', 26817030)
168
('nucleonics', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/NUCLEONICS.json', 26612298)
169
('appia', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/APPIA.json', 26272341)
170
('vudu', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/VUDU.json', 25924251)
171
('poynt', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/POYNT.json', 25923936)
172
('veracode', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/VERACODE.json', 25683875)
173
('fogger', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/FOGGER.json', 25641203)
174
('bizo',

('kifi', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/KIFI.json', 20557359)
248
('twyla', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/TWYLA.json', 20516970)
249
('inturn', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/INTURN.json', 20442874)
250
('serverless', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/SERVERLESS.json', 20374743)
251
('ario', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/ARIO.json', 20373702)
252
('enocean', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/ENOCEAN.json', 20311497)
253
('zimbra', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/ZIMBRA.json', 20192625)
254
('atheer', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/ATHEER.json', 20186760)
255
('monki', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/MONKI.json', 20083900)
256
('rignet', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/RIGNET.json', 20043860)
257
('issuu', '/Users/Arte

('prezi', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/PREZI.json', 16806971)
330
('ovascience', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/OVASCIENCE.json', 16756244)
331
('elastica', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/ELASTICA.json', 16748954)
332
('smartasset', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/SMARTASSET.json', 16618415)
333
('zoomin', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/ZOOMIN.json', 16614635)
334
('pureplay', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/PUREPLAY.json', 16594807)
335
('birst', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/BIRST.json', 16563599)
336
('mitro', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/MITRO.json', 16543891)
337
('hux', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/HUX.json', 16524469)
338
('automattic', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/AUTOMATTIC.json', 16480606)
339


...

('meegenius', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/MEEGENIUS.json', 783596)
3749
('leveleleven', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/LEVELELEVEN.json', 783079)
3750
('symtavision', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/SYMTAVISION.json', 782369)
3751
('rallyon', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/RALLYON.json', 781776)
3752
('cityvoter', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/CITYVOTER.json', 781653)
3753
('spingo', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/SPINGO.json', 781005)
3754
('sensorberg', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/SENSORBERG.json', 779683)
3755
('weplay', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/WEPLAY.json', 779252)
3756
('cotap', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/COTAP.json', 778493)
3757
('apnicure', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/APNICURE.json',

('breathometer', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/BREATHOMETER.json', 742668)
3831
('conduco', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/CONDUCO.json', 742563)
3832
('keymetrics', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/KEYMETRICS.json', 741450)
3833
('diffon', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/DIFFON.json', 740840)
3834
('netradyne', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/NETRADYNE.json', 739228)
3835
('cloudcheckr', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/CLOUDCHECKR.json', 738458)
3836
('netomat', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/NETOMAT.json', 738374)
3837
('graphicly', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/GRAPHICLY.json', 737826)
3838
('pheedo', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/PHEEDO.json', 737766)
3839
('liquidpiston', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/LIQUI

('pullstring', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/PULLSTRING.json', 706502)
3912
('pressmart', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/PRESSMART.json', 705867)
3913
('mindsnacks', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/MINDSNACKS.json', 705292)
3914
('snappytv', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/SNAPPYTV.json', 705036)
3915
('rocketon', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/ROCKETON.json', 704482)
3916
('blockcypher', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/BLOCKCYPHER.json', 703989)
3917
('evocatal', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/EVOCATAL.json', 703840)
3918
('nxthera', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/NXTHERA.json', 703804)
3919
('microf', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/MICROF.json', 703622)
3920
('cardiodiagnostics', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/

('velomedix', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/VELOMEDIX.json', 666794)
3993
('groxis', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/GROXIS.json', 666727)
3994
('bloomboard', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/BLOOMBOARD.json', 666455)
3995
('sonarmed', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/SONARMED.json', 666350)
3996
('watchmaster', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/WATCHMASTER.json', 666047)
3997
('picwell', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/PICWELL.json', 664860)
3998
('contentful', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/CONTENTFUL.json', 663853)
3999
('sebacia', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/SEBACIA.json', 662776)
4000
('zuberance', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/ZUBERANCE.json', 662619)
4001
('pacinian', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/PACINIAN.

('travelbank', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/TRAVELBANK.json', 633814)
4074
('parktag', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/PARKTAG.json', 633505)
4075
('littlstar', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/LITTLSTAR.json', 633474)
4076
('wochit', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/WOCHIT.json', 633094)
4077
('automile', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/AUTOMILE.json', 633085)
4078
('fashism', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/FASHISM.json', 633051)
4079
('genomind', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/GENOMIND.json', 632360)
4080
('neighbormd', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/NEIGHBORMD.json', 631449)
4081
('siliconstor', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/SILICONSTOR.json', 631372)
4082
('telnexus', '/Users/Artemiy/Disk-O/tyomkolton@mail.ru-mailru/ML/News_1/TELNEXUS.js

In [170]:
pycorpora.get_file("technology", "computer_sciences")

{'description': 'names of technologies related to computer science',
 'computer_sciences': ['.QL',
  'ActionScript',
  'ActiveRecord',
  'AIM',
  'Ajax',
  'Algol',
  'Amazon',
  'Angular',
  'Apache',
  'AppleScript',
  'AppStream',
  'ASPnet',
  'AutoHotKey',
  'Aviato',
  'AWS',
  'Backbone',
  'BASIC',
  'Bootstrap',
  'Bower',
  'Browserify',
  'Bundler',
  'C',
  'Csharp',
  'C+',
  'C++',
  'Canvas',
  'Capistrano',
  'Cassandra',
  'ClearDB',
  'CloudFormation',
  'CloudFront',
  'CloudSearch',
  'CloudTrail',
  'CloudWatch',
  'COBOL',
  'CodeCommit',
  'CodeDeploy',
  'CodePipeline',
  'CoffeeScript',
  'Cognito',
  'CouchDB',
  'CrunchBang',
  'CSS3',
  'Cucumber',
  'D3',
  'Dart',
  'Diaspora',
  'Discourse',
  'Django',
  'Drupal',
  'DynamoDB',
  'EBS',
  'EC2',
  'EJS',
  'ElacticBeanstalk',
  'Elasticache',
  'ElasticSearch',
  'Emacs',
  'Ember',
  'ERB',
  'Erlang',
  'Express',
  'Facebook',
  'Fedora',
  'Foursquare',
  'Flash',
  'Flickr',
  'FORTRAN',
  'Foundati

In [148]:
print(str(len(all_articles/num_articles)) + "% of articles were are containing corresponding keyphrases extracted from the descriptions of the companies")

0% of articles were are containing corresponding keyphrases extracted from the descriptions of the companies


In [ ]:
# Save dataset
if not os.path.exists(os.path.join(basedir, 'temp_data')):
    os.makedirs(os.path.join(basedir, 'temp_data'))
all_articles.to_csv(os.path.join(basedir, 'temp_data', 'all_articles.csv'), sep=";")
all_articles.head()

In [20]:
print(companies_list)

['ABRA', 'ACRONIS', 'ACTON', 'AIRSTONE', 'ALBERT', 'ALEA', 'ALIANZA', 'AMARA', 'ANDO', 'ARKIN', 'ARTSPACE', 'ASANA', 'ASOKA', 'ASTRID', 'ATHOS', 'AVA', 'AYR', 'BACKDOOR', 'BANNERMAN', 'BARKLY', 'BASTILLE', 'BBB', 'BEBO', 'BENTO', 'BIA', 'BINDO', 'BIOME', 'BIRDBOX', 'BITTORRENT', 'BIX', 'BIZEN', 'BOARDVOTE', 'BONOBOS', 'BOTTLENOSE', 'BOWERY', 'BRAINTREE', 'BRIT', 'BUDDYTV', 'CAIS', 'CALERA', 'CALVIN', 'CARTESIAN', 'CASPER', 'CASSATT', 'CENTRO', 'CERBERUS', 'CHAI', 'CLARK', 'CLEO', 'CLICKABLE', 'CLOE', 'COTOPAXI', 'CRONO', 'CROSSFADER', 'DAILYMOTION', 'DELOS', 'DIGBY', 'DIY', 'DOO', 'DOPPELGANGER', 'ECO', 'ELASTICA', 'ELEMENTUM', 'ELLIE', 'ELɘ', 'EMMA', 'ENDORPHIN', 'EPIPHYTE', 'EPS', 'ESS', 'EVERLY', 'EXCALIBUR', 'EXOGENESIS', 'EYESPOT', 'FAB', 'FACEBOOK', 'FANBASE', 'FARADAY', 'FIDELIS', 'FLASHPOINT', 'FLAVOUR', 'FLYER', 'FLYNN', 'FONDU', 'FORMSPRING', 'FRONTO', 'FRS', 'FULHAM', 'FUZE', 'FY', 'GAMETIME', 'GAMEZONE', 'GAMGEE', 'GELI', 'GENI', 'GOLDSTAR', 'GOLGI', 'GORGIAS', 'GRO', 'HALL

In [56]:
len(articles)

76920

In [175]:
all_articles.tail()

,UNPARSED,autor,byline,contact,content,dateline,distribution,error,error_code,grafic,...,pub_type,rubrik,section,series,source,company,uuid,description,country,correction_date
4395,"AFX - Asia\n\n November 2,...",NaN,NaN,NaN,"AFX - Asia\n\n November 2,...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,musx,9d2a0961-a4dc-ae6e-886b-64246ca3f4ab,"Whenever you first meet someone, there is the ...",NaN,NaN
4396,"AFX - Asia\n\n August 16,...",NaN,NaN,NaN,"AFX - Asia\n\n August 16,...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,musx,9d2a0961-a4dc-ae6e-886b-64246ca3f4ab,"Whenever you first meet someone, there is the ...",NaN,NaN
4397,Manila Times\n\n J...,NaN,NaN,NaN,Manila Times\n\n J...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,musx,9d2a0961-a4dc-ae6e-886b-64246ca3f4ab,"Whenever you first meet someone, there is the ...",NaN,NaN
4398,"AFX - Asia\n\n May 24, 2...",NaN,NaN,NaN,"AFX - Asia\n\n May 24, 2...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,musx,9d2a0961-a4dc-ae6e-886b-64246ca3f4ab,"Whenever you first meet someone, there is the ...",NaN,NaN
4399,"AFX - Asia\n\n April 22,...",NaN,NaN,NaN,"AFX - Asia\n\n April 22,...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,musx,9d2a0961-a4dc-ae6e-886b-64246ca3f4ab,"Whenever you first meet someone, there is the ...",NaN,NaN
